# Software Requirements Specification for Chemistry Code
## Samuel J. Crawford




# Table of Contents
<a id="Sec:ToC"></a>

An outline of all sections included in this SRS is recorded here for easy reference.

- [Table of Contents](#Sec:ToC)
- [Reference Material](#Sec:RefMat)
 - [Table of Symbols](#Sec:ToS)
 - [Abbreviations and Acronyms](#Sec:TAbbAcc)
- [Introduction](#Sec:Intro)
 - [Purpose of Document](#Sec:DocPurpose)
 - [Scope of Requirements](#Sec:ReqsScope)
 - [Characteristics of Intended Reader](#Sec:ReaderChars)
 - [Organization of Document](#Sec:DocOrg)
- [General System Description](#Sec:GenSysDesc)
 - [System Context](#Sec:SysContext)
 - [User Characteristics](#Sec:UserChars)
 - [System Constraints](#Sec:SysConstraints)
- [Specific System Description](#Sec:SpecSystDesc)
- [Problem Description](#Sec:ProbDesc)
 - [Terminology and Definitions](#Sec:TermDefs)
 - [Physical System Description](#Sec:PhysSyst)
 - [Goal Statements](#Sec:GoalStmt)
- [Solution Characteristics Specification](#Sec:SolCharSpec)
 - [Assumptions](#Sec:Assumps)
 - [Theoretical Models](#Sec:TMs)
 - [General Definitions](#Sec:GDs)
 - [Data Definitions](#Sec:DDs)
- [Requirements](#Sec:Requirements)
 - [Functional Requirements](#Sec:FRs)
 - [Non-Functional Requirements](#Sec:NFRs)
- [Likely Changes](#Sec:LCs)
- [Unlikely Changes](#Sec:UCs)
- [Traceability Matrices and Graphs](#Sec:TraceMatrices)
- [Values of Auxiliary Constants](#Sec:AuxConstants)
- [References](#Sec:References)

# Reference Material
<a id="Sec:RefMat"></a>

This section records information for easy reference.

## Table of Symbols
<a id="Sec:ToS"></a>

The symbols used in this document are summarized in the [Table of Symbols](#Table:ToS) along with their units. The symbols are listed in alphabetical order.

<a id="Table:ToS"></a>

|Symbol|Description|Units|
|:--- |:--- |:--- |
|$0$|Zero vector|--|
|$A$|Generic matrix|--|
|$b$|Generic vector|--|
|$c$|Generic vector|--|
|$E$|Element data type|--|
|$MAINTAIN_FRAC$|Maintainability fraction|--|
|$r$|Representation of a chemical equation|--|
|$x$|Generic vector|--|


## Abbreviations and Acronyms
<a id="Sec:TAbbAcc"></a>

<a id="Table:TAbbAcc"></a>

|Abbreviation|Full Form|
|:--- |:--- |
|A|Assumption|
|ChemCode|Chemistry Code|
|DD|Data Definition|
|IUPAC|International Union of Pure and Applied Chemistry|
|ODE|Ordinary Differential Equation|
|R|Requirement|
|SRS|Software Requirements Specification|
|TM|Theoretical Model|
|UC|Unlikely Change|


# Introduction
<a id="Sec:Intro"></a>

Chemical equations are common ways of representing chemical reactions ([Fig:physSysFig](#Figure:physSysFig) shows an example of a chemical equation). Subscripts indicate the number of atoms of each element present in the given chemical compound. A chemical equation is "balanced" if there are the same number of atoms of each element before and after the reaction takes place, which satisfies the Law of Conservation of Mass. Chemical equations are balanced by introducing coefficients before each chemical formula (this coefficient may be "1", in which case is implicit and not added to the equation). We want a tool to balance chemical equations automatically to improve the productivity of scientists and engineers and reduce the potential for human error. This program should balance a given chemical equation if it is feasible and if it is not, it should provide a message communicating this to the user. The program that performs these tasks as documented here will be called Chemistry Code (ChemCode).

The following section provides an overview of the Software Requirements Specification (SRS) for ChemCode. This section explains the purpose of this document, the scope of the requirements, the characteristics of the intended reader, and the organization of the document.

## Purpose of Document
<a id="Sec:DocPurpose"></a>

The primary purpose of this document is to record the requirements of the Chemistry Code. Goals, assumptions, theoretical models, definitions, and other model derivation information are specified, allowing the reader to fully understand and verify the purpose and scientific basis of ChemCode. With the exception of [system constraints](#Sec:SysConstraints), this SRS will remain abstract, describing what problem is being solved, but not how to solve it.

This document will be used as a starting point for subsequent development phases, including writing the design specification and the software verification and validation plan. The design document will show how the requirements are to be realized, including decisions on the numerical algorithms and programming environment. The verification and validation plan will show the steps that will be used to increase confidence in the software documentation and the implementation. Although the SRS fits in a series of documents that follow the so-called waterfall model, the actual development process is not constrained in any way. Even when the waterfall model is not followed, as Parnas and Clements point out <a href=#parnasClements1986>parnasClements1986</a> , the most logical way to present the documentation is still to "fake" a rational design process.

## Scope of Requirements
<a id="Sec:ReqsScope"></a>

The scope of the requirements includes all inputted chemical equations where the total number of chemical compounds is at most one more than the total number of elements. The scope also includes all inputted chemical formulas that describe real chemical compounds, are formatted following a set of conventions, and only consist of atomic symbols and subscripts.

## Characteristics of Intended Reader
<a id="Sec:ReaderChars"></a>

Reviewers of this documentation should have an understanding of high school chemistry (namely stochiometry) and third year linear optimization (namely integer programming). The users of ChemCode can have a lower level of expertise, as explained in [Sec:User Characteristics](#Sec:UserChars).

## Organization of Document
<a id="Sec:DocOrg"></a>

The organization of this document follows the template for an SRS for scientific computing software proposed by <a href=#koothoor2013>koothoor2013</a>  and <a href=#smithLai2005>smithLai2005</a> . The presentation follows the standard pattern of presenting goals, theories, definitions, and assumptions. For readers that would like a more bottom up approach, they can start reading the [instance models](#Sec:IMs) and trace back to find any additional information they require.

The [goal statements](#Sec:GoalStmt) are refined to the theoretical models and the [theoretical models](#Sec:TMs) to the [instance models](#Sec:IMs).

# General System Description
<a id="Sec:GenSysDesc"></a>

This section provides general information about the system. It identifies the interfaces between the system and its environment, describes the user characteristics, and lists the system constraints.

## System Context
<a id="Sec:SysContext"></a>

[Fig:sysCtxFig](#Figure:sysCtxFig) shows the system context. A circle represents an external entity outside the software, the user in this case. A rectangle represents the software system itself (ChemCode). Arrows are used to show the data flow between the system and its environment.
<a id="Figure:sysCtxFig"></a>
<figure>
<img src="../../../../datafiles/chemcode/sysCtxFig.png" alt="System Context" >
</figure>

The interaction between the product and the user is through a user interface. The responsibilities of the user and the system are as follows:

- User Responsibilities
 - Provide an unbalanced chemical equation, ensuring conformation to input data format required by ChemCode
 - Ensure required [software assumptions](#Sec:Assumps) are appropriate for the problem to which the user is applying the software
- ChemCode Responsibilities
 - Detect data type mismatch, such as a string of characters instead of a floating point number
 - Format the inputted chemical equation as a matrix
 - If the inputted chemical equation is feasible, find its balanced form with the smallest possible whole number coefficients
- External Library Responsibilities
 - Solve the integer linear programming problem for the inputted chemical equation

## User Characteristics
<a id="Sec:UserChars"></a>

The end user of ChemCode should have an understanding of high school chemistry (namely stochiometry).

## System Constraints
<a id="Sec:SysConstraints"></a>

ChemCode will be developed using Drasil <a href=#drasilSource>drasilSource</a> , "a framework for generating high-quality documentation and code for Scientific Computing Software" <a href=#maclachlan2021>maclachlan2021</a> , with the goal of extending it by adding concepts relevant to the problem outlined in the [problem description](#Sec:ProbDesc). Since Drasil is built on the idea of reusability, external libraries will be used to solve the integer programming problems. This was previously done with ordinary differential equation (ODE) solvers, since "creating a complete ODE solver in Drasil would take considerable time, and there are already many reliable external libraries ... tested by long use" <a href=#chen2022>chen2022</a> (pg. 24). These rationales also apply to ILP solvers.

# Specific System Description
<a id="Sec:SpecSystDesc"></a>

This section first presents the problem description, which gives a high-level view of the problem to be solved. This is followed by the solution characteristics specification, which presents the assumptions, theories, and definitions that are used.

## Problem Description
<a id="Sec:ProbDesc"></a>

A system is needed to balance chemical equations so they can be useful for other computations <a href=#lund2023>lund2023</a> . Additionally, since molecules only exist in positive integer quantities (since dividing a molecule changes it into different molecules), the coefficients used to balance the chemical equation should be whole numbers, and by convention should be as small as possible <a href=#lund2023>lund2023</a> . There are some cases where the coefficients are not integers (see <a href=#nonIntCoeffSource>nonIntCoeffSource</a> ), but this is not in the scope of this program.

### Terminology and Definitions
<a id="Sec:TermDefs"></a>

This subsection provides a list of terms that are used in the subsequent sections and their meaning, with the purpose of reducing ambiguity and making it easier to correctly understand the requirements.

- Compound: A molecule made up of more than one atom, which may or may not be of different elements.
- Element: The group of all atoms with the same number of protons in the atomic nucleus. For example, all atoms with one proton are hydrogen atoms.
- Equation: A textual representation of a chemical reaction.
- Hydrate: "a compound formed by the chemical combination of water and some other substance in a definite molecular ratio" <a href=#hydrateSource>hydrateSource</a> .
- Isotope: An atom that is the same element as another but has a different number of neutrons <a href=#lund2023>lund2023</a> .
- Polyatomic ion: A group of atoms "bonded together that carr[ies] an overall electric charge" <a href=#lund2023>lund2023</a> .
- Polymer: A macromolecule "formed by the chemical bonding of large numbers of smaller molecules" <a href=#polymerSource>polymerSource</a> .
- Product: A substance formed by a chemical reaction.
- Reactant: A substance involved in and changed by a chemical reaction.
- Reaction: An interaction between different types of matter that results in at least one new substance being formed <a href=#lund2023>lund2023</a> .

### Physical System Description
<a id="Sec:PhysSyst"></a>

The physical system of ChemCode, as shown in [Fig:physSysFig](#Figure:physSysFig), includes the following elements:

PS1: The reactants of a given chemical reaction.

PS2: The products of the same chemical reaction.

<a id="Figure:physSysFig"></a>
<figure>
<img src="../../../../datafiles/chemcode/physSysFig.png" alt="Physical System Description" >
</figure>

### Goal Statements
<a id="Sec:GoalStmt"></a>

Given a representation of a chemical equation, the goal statements are:

<div id=\"findBalancedForm\">
findBalancedForm: Find its balanced form with the smallest positive integer coefficients if it is feasible.

</div>

## Solution Characteristics Specification
<a id="Sec:SolCharSpec"></a>

The instance models that govern ChemCode are presented in the [Instance Model Section](#Sec:IMs). The information to understand the meaning of the instance models and their derivation is also presented, so that the instance models can be verified.

### Assumptions
<a id="Sec:Assumps"></a>

This section simplifies the original problem and helps in developing the theoretical models by filling in the missing information for the physical system. The assumptions refine the scope by providing more detail.

<div id=\"elemCompDiff\">
elemCompDiff: For all chemical equations, the total number of chemical compounds is at most one more than the total number of elements. (RefBy: [UC:allEqsPermitted](#allEqsPermitted).)

</div>
<div id=\"validForms\">
validForms: All inputted chemical formulas describe real chemical compounds. (RefBy: [UC:checkValidForms](#checkValidForms).)

</div>
<div id=\"validEqns\">
validEqns: All inputted chemical equations describe real chemical reactions. (RefBy: [UC:checkValidEqns](#checkValidEqns).)

</div>
<div id=\"correctInputFormat\">
correctInputFormat: All inputted chemical formulas follow the conventions laid out in <a href=#inorganicIUPAC>inorganicIUPAC</a>  and <a href=#organicIUPAC>organicIUPAC</a>  by the International Union of Pure and Applied Chemistry (IUPAC). (RefBy: [LC:incorrectInputFormat](#incorrectInputFormat).)

</div>
<div id=\"simpleForms\">
simpleForms: All inputted chemical formulas only consist of atomic symbols and subscripts. This means that they cannot contain dots, parentheses, hyphens, or superscripts, so hydrates, polymers, and formulas with isotopes and some with polyatomic ions will not be present in inputted chemical equations. (RefBy: [LC:complexForms](#complexForms).)

</div>

### Theoretical Models
<a id="Sec:TMs"></a>

This section focuses on the general equations and laws that ChemCode is based on.
<br>
<a id="TM:intLinProg"></a>
<table class=\"tdefn\">
<tr>
<th>Refname</th>
<td>
<b>TM:intLinProg</b>
</td>
</tr>
<tr>
<th>Label</th>
<td>

Integer linear program
</td>
</tr>
<tr>
<th>Equation</th>
<td>
$$\begin{cases}\symbf{c} \symbf{x}, & 1\\\symbf{A} \symbf{x}\leq{}\symbf{b}, & 2\\\symbf{x}\geq{}\symbf{0}, & 3\end{cases}$$
</td>
</tr>
<tr>
<th>Description</th>
<td>

<li>$c$ is the generic vector (Unitless)</li>
<li>$x$ is the generic vector (Unitless)</li>
<li>$A$ is the generic matrix (Unitless)</li>
<li>$b$ is the generic vector (Unitless)</li>
<li>$0$ is the zero vector (Unitless)</li>
</td>
</tr>
<tr>
<th>Notes</th>
<td>

The above equation gives the canonical form of an integer linear program, which is "a mathematical optimization or feasibility program in which some or all of the variables are restricted to be integers [and] the objective function and the constraints (other than the integer constraints) are linear" <a href=#ilpWiki>ilpWiki</a> .

The values of $x$ are unknown and will be solved for.
</td>
</tr>
<tr>
<th>Source</th>
<td>

<a href=#ilpWiki>ilpWiki</a> 
</td>
</tr>
<tr>
<th>RefBy</th>
<td>


</td>
</tr>
</table>

### General Definitions
<a id="Sec:GDs"></a>

There are no general definitions.

### Data Definitions
<a id="Sec:DDs"></a>

This section collects and defines all the data needed to build the instance models.
<br>
<a id="DD:elementType"></a>
<table class=\"ddefn\">
<tr>
<th>Refname</th>
<td>
<b>DD:elementType</b>
</td>
</tr>
<tr>
<th>Label</th>
<td>

Element data type
</td>
</tr>
<tr>
<th>Symbol</th>
<td>

$E$
</td>
</tr>
<tr>
<th>Equation</th>
<td>
$$E=E$$
</td>
</tr>
<tr>
<th>Description</th>
<td>

<li>$E$ is the element data type (Unitless)</li>
<li>$E$ is the element data type (Unitless)</li>
</td>
</tr>
<tr>
<th>Notes</th>
<td>

A type representing each element from <a href=#elemListWiki>elemListWiki</a> .
</td>
</tr>
<tr>
<th>Source</th>
<td>

<a href=#smithChemSpec>smithChemSpec</a> 
</td>
</tr>
<tr>
<th>RefBy</th>
<td>


</td>
</tr>
</table>

# Requirements
<a id="Sec:Requirements"></a>

This section provides the functional requirements, the tasks and behaviours that the software is expected to complete, and the non-functional requirements, the qualities that the software is expected to exhibit.

## Functional Requirements
<a id="Sec:FRs"></a>

This section provides the functional requirements, the tasks and behaviours that the software is expected to complete.

<div id=\"inputValues\">
Input-Values: Input the values from [Tab:ReqInputs](#Table:ReqInputs).

</div>
<div id=\"convertMatrix\">
Convert-to-Matrix: Convert the inputted chemical equation to matrix form.

</div>
<div id=\"feasOut\">
Feasible-Output: If the inputted chemical equation is feasible, output its balanced form with the smallest positive integer coefficients possible.

</div>
<div id=\"infeasOut\">
Infeasible-Output: If the inputted chemical equation is infeasible, output a descriptive message.

</div>

<a id="Table:ReqInputs"></a>

|Symbol|Description|Units|
|:--- |:--- |:--- |
|$r$|Representation of a chemical equation|--|


## Non-Functional Requirements
<a id="Sec:NFRs"></a>

This section provides the non-functional requirements, the qualities that the software is expected to exhibit.

<div id=\"accurate\">
Accurate: Chemical equations are only useful if they are balanced <a href=#lund2023>lund2023</a> , so computed coefficients should be exact.

</div>
<div id=\"verifiable\">
Verifiable: The code is tested following the Verification and Validation (VnV) Plan.

</div>
<div id=\"understandable\">
Understandable: A new intended user (as described by [Sec:User Characteristics](#Sec:UserChars)) is able to learn how to use ChemCode in an acceptable amount of time, as measured by the procedure in Section 10 of the VnV Plan.

</div>
<div id=\"usable\">
Usable: An intended user (as described by [Sec:User Characteristics](#Sec:UserChars)) finds ChemCode easy to use, as measured by the procedure in Section 11 of the VnV Plan.

</div>
<div id=\"reusable\">
Reusable: The code is modularized.

</div>
<div id=\"maintainable\">
Maintainable: The development time for any of the likely changes will not exceed $MAINTAIN_FRAC$ of the original development time.

</div>
<div id=\"portable\">
Portable: The code is able to be run on systems with the corresponding programming language installed, including systems running on Windows or macOS. The tests from the VnV Plan should pass in these environments.

</div>

# Likely Changes
<a id="Sec:LCs"></a>

This section lists the likely changes to be made to the software.

<div id=\"complexForms\">
complexForms: [A:simpleForms](#simpleForms) assumes that inputted chemical formulas only consist of atomic symbols and subscripts. In the future, the user might be able to input more complex chemical formulas, such as those containing dots, parentheses, hyphens, or superscripts.

</div>
<div id=\"calcMoles\">
calcMoles: In the future, ChemCode might be able to, given the amount of one substance (in moles) in a reaction, calculate the amount of every other substance (also in moles) required/produced by the reaction.

</div>
<div id=\"detLimReag\">
detLimReag: In the future, ChemCode might be able to, given the amount of more than one reactant (in moles) in a reaction, determine the limiting reactant(s). This is dependent on [LC:calcMoles](#calcMoles).

</div>
<div id=\"calcYield\">
calcYield: In the future, ChemCode might be able to, given the amount of more than one reactant (in moles) in a reaction, calculate the theoretical yield of each product (also in moles). This is dependent on [LC:detLimReag](#detLimReag).

</div>
<div id=\"calcExcess\">
calcExcess: In the future, ChemCode might be able to, given the amount of more than one reactant (in moles) in a reaction, calculate the amount of excess reactant(s) (also in moles). This is dependent on [LC:detLimReag](#detLimReag).

</div>
<div id=\"incorrectInputFormat\">
incorrectInputFormat: [A:correctInputFormat](#correctInputFormat) assumes that inputted chemical formulas follow the conventions laid out in <a href=#inorganicIUPAC>inorganicIUPAC</a>  and <a href=#organicIUPAC>organicIUPAC</a> . In the future, ChemCode might be able to parse valid inputted chemical formulas that do not follow these conventions and format them correctly when outputting them.

</div>
<div id=\"identifyPhaseLabels\">
identifyPhaseLabels: In the future, ChemCode might be able to, given the phase labels for the reactants of a chemical reaction, identify the phase labels for the products, which would involve determining solubility.

</div>

# Unlikely Changes
<a id="Sec:UCs"></a>

This section lists the unlikely changes to be made to the software.

<div id=\"allEqsPermitted\">
allEqsPermitted: [A:elemCompDiff](#elemCompDiff) assumes...

</div>
<div id=\"checkValidForms\">
checkValidForms: [A:validForms](#validForms) assumes...

</div>
<div id=\"checkValidEqns\">
checkValidEqns: [A:validEqns](#validEqns) assumes...

</div>

# Traceability Matrices and Graphs
<a id="Sec:TraceMatrices"></a>

The purpose of the traceability matrices is to provide easy references on what has to be additionally modified if a certain component is changed. Every time a component is changed, the items in the column of that component that are marked with an "X" should be modified as well. [Tab:TraceMatAvsA](#Table:TraceMatAvsA) shows the dependencies of assumptions on the assumptions. [Tab:TraceMatAvsAll](#Table:TraceMatAvsAll) shows the dependencies of data definitions, theoretical models, general definitions, instance models, requirements, likely changes, and unlikely changes on the assumptions. [Tab:TraceMatRefvsRef](#Table:TraceMatRefvsRef) shows the dependencies of data definitions, theoretical models, general definitions, and instance models with each other. [Tab:TraceMatAllvsR](#Table:TraceMatAllvsR) shows the dependencies of requirements, goal statements on the data definitions, theoretical models, general definitions, and instance models.

<a id="Table:TraceMatAvsA"></a>

||[A:elemCompDiff](#elemCompDiff)|[A:validForms](#validForms)|[A:validEqns](#validEqns)|[A:correctInputFormat](#correctInputFormat)|[A:simpleForms](#simpleForms)|
|:--- |:--- |:--- |:--- |:--- |:--- |
|[A:elemCompDiff](#elemCompDiff)||||||
|[A:validForms](#validForms)||||||
|[A:validEqns](#validEqns)||||||
|[A:correctInputFormat](#correctInputFormat)||||||
|[A:simpleForms](#simpleForms)||||||


<a id="Table:TraceMatAvsAll"></a>

||[A:elemCompDiff](#elemCompDiff)|[A:validForms](#validForms)|[A:validEqns](#validEqns)|[A:correctInputFormat](#correctInputFormat)|[A:simpleForms](#simpleForms)|
|:--- |:--- |:--- |:--- |:--- |:--- |
|[DD:elementType](#DD:elementType)||||||
|[TM:intLinProg](#TM:intLinProg)||||||
|[FR:Input-Values](#inputValues)||||||
|[FR:Convert-to-Matrix](#convertMatrix)||||||
|[FR:Feasible-Output](#feasOut)||||||
|[FR:Infeasible-Output](#infeasOut)||||||
|[NFR:Accurate](#accurate)||||||
|[NFR:Verifiable](#verifiable)||||||
|[NFR:Understandable](#understandable)||||||
|[NFR:Usable](#usable)||||||
|[NFR:Reusable](#reusable)||||||
|[NFR:Maintainable](#maintainable)||||||
|[NFR:Portable](#portable)||||||
|[LC:complexForms](#complexForms)|||||X|
|[LC:calcMoles](#calcMoles)||||||
|[LC:detLimReag](#detLimReag)||||||
|[LC:calcYield](#calcYield)||||||
|[LC:calcExcess](#calcExcess)||||||
|[LC:incorrectInputFormat](#incorrectInputFormat)||||X||
|[LC:identifyPhaseLabels](#identifyPhaseLabels)||||||
|[UC:allEqsPermitted](#allEqsPermitted)|X|||||
|[UC:checkValidForms](#checkValidForms)||X||||
|[UC:checkValidEqns](#checkValidEqns)|||X|||


<a id="Table:TraceMatRefvsRef"></a>

||[DD:elementType](#DD:elementType)|[TM:intLinProg](#TM:intLinProg)|
|:--- |:--- |:--- |
|[DD:elementType](#DD:elementType)|||
|[TM:intLinProg](#TM:intLinProg)|||


<a id="Table:TraceMatAllvsR"></a>

||[DD:elementType](#DD:elementType)|[TM:intLinProg](#TM:intLinProg)|[FR:Input-Values](#inputValues)|[FR:Convert-to-Matrix](#convertMatrix)|[FR:Feasible-Output](#feasOut)|[FR:Infeasible-Output](#infeasOut)|[NFR:Accurate](#accurate)|[NFR:Verifiable](#verifiable)|[NFR:Understandable](#understandable)|[NFR:Usable](#usable)|[NFR:Reusable](#reusable)|[NFR:Maintainable](#maintainable)|[NFR:Portable](#portable)|
|:--- |:--- |:--- |:--- |:--- |:--- |:--- |:--- |:--- |:--- |:--- |:--- |:--- |:--- |
|[GS:findBalancedForm](#findBalancedForm)||||||||||||||
|[FR:Input-Values](#inputValues)||||||||||||||
|[FR:Convert-to-Matrix](#convertMatrix)||||||||||||||
|[FR:Feasible-Output](#feasOut)||||||||||||||
|[FR:Infeasible-Output](#infeasOut)||||||||||||||
|[NFR:Accurate](#accurate)||||||||||||||
|[NFR:Verifiable](#verifiable)||||||||||||||
|[NFR:Understandable](#understandable)||||||||||||||
|[NFR:Usable](#usable)||||||||||||||
|[NFR:Reusable](#reusable)||||||||||||||
|[NFR:Maintainable](#maintainable)||||||||||||||
|[NFR:Portable](#portable)||||||||||||||


The purpose of the traceability graphs is also to provide easy references on what has to be additionally modified if a certain component is changed. The arrows in the graphs represent dependencies. The component at the tail of an arrow is depended on by the component at the head of that arrow. Therefore, if a component is changed, the components that it points to should also be changed. [Fig:TraceGraphAvsA](#Figure:TraceGraphAvsA) shows the dependencies of assumptions on the assumptions. [Fig:TraceGraphAvsAll](#Figure:TraceGraphAvsAll) shows the dependencies of data definitions, theoretical models, general definitions, instance models, requirements, likely changes, and unlikely changes on the assumptions. [Fig:TraceGraphRefvsRef](#Figure:TraceGraphRefvsRef) shows the dependencies of data definitions, theoretical models, general definitions, and instance models with each other. [Fig:TraceGraphAllvsR](#Figure:TraceGraphAllvsR) shows the dependencies of requirements, goal statements on the data definitions, theoretical models, general definitions, and instance models. [Fig:TraceGraphAllvsAll](#Figure:TraceGraphAllvsAll) shows the dependencies of dependencies of assumptions, models, definitions, requirements, goals, and changes with each other.
<a id="Figure:TraceGraphAvsA"></a>
<figure>
<img src="../../../../traceygraphs/chemcode/avsa.svg" alt="TraceGraphAvsA" >
</figure>
<a id="Figure:TraceGraphAvsAll"></a>
<figure>
<img src="../../../../traceygraphs/chemcode/avsall.svg" alt="TraceGraphAvsAll" >
</figure>
<a id="Figure:TraceGraphRefvsRef"></a>
<figure>
<img src="../../../../traceygraphs/chemcode/refvsref.svg" alt="TraceGraphRefvsRef" >
</figure>
<a id="Figure:TraceGraphAllvsR"></a>
<figure>
<img src="../../../../traceygraphs/chemcode/allvsr.svg" alt="TraceGraphAllvsR" >
</figure>
<a id="Figure:TraceGraphAllvsAll"></a>
<figure>
<img src="../../../../traceygraphs/chemcode/allvsall.svg" alt="TraceGraphAllvsAll" >
</figure>

For convenience, the following graphs can be found at the links below:

- <a href=\"../../../../traceygraphs/chemcode/avsa.svg\">TraceGraphAvsA</a>
- <a href=\"../../../../traceygraphs/chemcode/avsall.svg\">TraceGraphAvsAll</a>
- <a href=\"../../../../traceygraphs/chemcode/refvsref.svg\">TraceGraphRefvsRef</a>
- <a href=\"../../../../traceygraphs/chemcode/allvsr.svg\">TraceGraphAllvsR</a>
- <a href=\"../../../../traceygraphs/chemcode/allvsall.svg\">TraceGraphAllvsAll</a>

# Values of Auxiliary Constants
<a id="Sec:AuxConstants"></a>

This section contains the standard values that are used for calculations in ChemCode.

<a id="Table:TAuxConsts"></a>

|Symbol|Description|Value|Unit|
|:--- |:--- |:--- |:--- |
|$MAINTAIN_FRAC$|maintainability fraction|$10.0%$|--|


# References
<a id="Sec:References"></a>
<a id="nonIntCoeffSource"></a>
[1]: U. S. Department of the Interior, Bergholm, A. "Oxidation of Pyrite." Boulder, CO, USA: 1995.
<a id="drasilSource"></a>
[2]: Carette, Jacques, Smith, W. Spencer, Balaci, Jason, Wu, Ting-Yu, Crawford, Samuel J., Chen, Dong, Szymczak, Dan, MacLachlan, Brooks, Scime, Dan, and Miazi, Maryyam. <em>Drasil</em>. February, 2021. <a href="https://jacquescarette.github.io/Drasil/">https://jacquescarette.github.io/Drasil/</a>.
<a id="chen2022"></a>
[3]: Chen, Dong. <em>Solving Higher-Order ODEs in Drasil</em>. Edited by Jacques Carette and W. Spencer Smith, McMaster University, Hamilton, ON, Canada: September, 2022. <a href="https://github.com/JacquesCarette/Drasil/blob/master/People/Dong/Thesis_Main.pdf">https://github.com/JacquesCarette/Drasil/blob/master/People/Dong/Thesis_Main.pdf</a>. Print.
<a id="hydrateSource"></a>
[4]: HarperCollins Publishers. <em>hydrate</em>. <em>Collins</em>, New York, NY, USA: HarperCollins,
<a id="inorganicIUPAC"></a>
[5]: International Union of Pure and Applied Chemistry, Connelly, Neil G., Damhus, Ture, Hartshorn, Richard M., and Hutton, Alan T. <em>Nomenclature of Inorganic Chemistry: IUPAC Recommendations 2005</em>. Cambridge, UK: The Royal Society of Chemistry, 2005. <a href="https://iupac.org/wp-content/uploads/2016/07/Red_Book_2005.pdf">https://iupac.org/wp-content/uploads/2016/07/Red_Book_2005.pdf</a>. Print.
<a id="organicIUPAC"></a>
[6]: International Union of Pure and Applied Chemistry, Moss, G. P., Favre, Henri A., and Powell, Warren H. <em>Nomenclature of Organic Chemistry: IUPAC Recommendations and Preferred Names 2013</em>. Cambridge, UK: The Royal Society of Chemistry, 2013. <a href="https://iupac.qmul.ac.uk/BlueBook/PDF/">https://iupac.qmul.ac.uk/BlueBook/PDF/</a>. Print.
<a id="koothoor2013"></a>
[7]: Koothoor, Nirmitha. <em>A document drive approach to certifying scientific computing software</em>. McMaster University, Hamilton, ON, Canada: 2013. Print.
<a id="lund2023"></a>
[8]: Lund, Lance. <em>Introduction to Chemistry</em>. Cambridge and Coon Rapids, MN, USA: LibreTexts, January, 2023. <a href="https://chem.libretexts.org/Courses/Anoka-Ramsey_Community_College/Introduction_to_Chemistry">https://chem.libretexts.org/Courses/Anoka-Ramsey_Community_College/Introduction_to_Chemistry</a>. Print.
<a id="maclachlan2021"></a>
[9]: MacLachlan, Brooks. <em>A Design Language for Scientific Computing Software in Drasil</em>. Edited by Jacques Carette and W. Spencer Smith, McMaster University, Hamilton, ON, Canada: November, 2021. <a href="https://macsphere.mcmaster.ca/bitstream/11375/25542/2/maclachlan_brooks_2020july_masc.pdf">https://macsphere.mcmaster.ca/bitstream/11375/25542/2/maclachlan_brooks_2020july_masc.pdf</a>. Print.
<a id="parnasClements1986"></a>
[10]: Parnas, David L. and Clements, P. C. "A rational design process: How and why to fake it." <em>IEEE Transactions on Software Engineering</em>, vol. 12, no. 2, Washington, USA: February, 1986. pp. 251&ndash;257. Print.
<a id="smithChemSpec"></a>
[11]: Smith, W. Spencer. <em>Assignment 2</em>. February, 2020. <a href="https://gitlab.cas.mcmaster.ca/smiths/se2aa4_cs2me3/-/blob/master/Assignments/PreviousYears/2020/A2-ChemReacts/A2.pdf">https://gitlab.cas.mcmaster.ca/smiths/se2aa4_cs2me3/-/blob/master/Assignments/PreviousYears/2020/A2-ChemReacts/A2.pdf</a>.
<a id="smithLai2005"></a>
[12]: Smith, W. Spencer and Lai, Lei. "A new requirements template for scientific computing." <em>Proceedings of the First International Workshop on Situational Requirements Engineering Processes - Methods, Techniques and Tools to Support Situation-Specific Requirements Engineering Processes, SREP'05</em>. Edited by PJ Agerfalk, N. Kraiem, and J. Ralyte, Paris, France: 2005. pp. 107&ndash;121. In conjunction with 13th IEEE International Requirements Engineering Conference,
<a id="polymerSource"></a>
[13]: Wexler, Philip. <em>Encyclopedia of Toxicology</em>. 3st. ed., Amsterdam, NL: Academic Press, 2014. Print.
<a id="ilpWiki"></a>
[14]: Wikipedia Contributors. <em>Integer programming</em>. March, 2023. <a href="https://en.wikipedia.org/wiki/Integer_programming">https://en.wikipedia.org/wiki/Integer_programming</a>.
<a id="elemListWiki"></a>
[15]: Wikipedia Contributors. <em>List of chemical elements</em>. January, 2023. <a href="https://en.wikipedia.org/wiki/List_of_chemical_elements">https://en.wikipedia.org/wiki/List_of_chemical_elements</a>.

